In [ ]:
import torchaudio
fpath = "/graft2/datasets/znovack/LJSpeech-1.1/wavs/"
# get length of each file in fpath
import os
import tqdm


lengths = {}
for f in tqdm.tqdm(os.listdir(fpath)):
    if f.endswith(".wav"):
        info = torchaudio.info(os.path.join(fpath, f))
        print(f, info.num_frames)
        lengths[f] = info.num_frames

# write to json
import json
with open("ljspeech_info.json", "w") as f:
    json.dump(lengths, f)

In [ ]:
# for given path, get length, sampling rate, and bit depth
import torchaudio
import os
import tqdm
def get_wav_info(path):
    # get length, sr, bit depth, and # of chanels
    info = torchaudio.info(path)
    return info.num_frames, info.sample_rate, info.bits_per_sample, info.num_channels

pth = '/graft3/datasets/pnlong/lnac/sashimi/data/musdb18stereo/valid/'
wav_info = {}
for root, _, files in tqdm.tqdm(os.walk(pth)):
    for f in files:
        if f.lower().endswith(('.flac', '.wav')):
            full_path = os.path.join(root, f)
            try:
                length, sr, bps, ch = get_wav_info(full_path)
            except Exception as e:
                print(f"Error reading {full_path}: {e}")
                continue
            key = os.path.basename(full_path)
            wav_info[key] = {'length': length, 'sample_rate': sr, 'bits_per_sample': bps, 'n_channels': ch}
            
import json
with open('musdb18stereo_valid_info.json', 'w') as f:
    json.dump(wav_info, f)

In [3]:
import json

# load
with open('musdb18stereo_valid_info.json', 'r') as f:
    wav_info = json.load(f)

# get statistics
lengths = [v['length'] for v in wav_info.values()]
sample_rates = [v['sample_rate'] for v in wav_info.values()]
bit_depths = [v['bits_per_sample'] for v in wav_info.values()]
channels = [v['n_channels'] for v in wav_info.values()]
import numpy as np
print("Length: min {}, max {}, mean {}, std {}".format(np.min(lengths), np.max(lengths), np.mean(lengths), np.std(lengths)))
print("Sample Rate: min {}, max {}, mean {}, std {}".format(np.min(sample_rates), np.max(sample_rates), np.mean(sample_rates), np.std(sample_rates)))
print("Bit Depth: min {}, max {}, mean {}, std {}".format(np.min(bit_depths), np.max(bit_depths), np.mean(bit_depths), np.std(bit_depths)))
print("Channels: min {}, max {}, mean {}, std {}".format(np.min(channels), np.max(channels), np.mean(channels), np.std(channels)))

# get count of unique sample rates, bit depths, and channels
from collections import Counter
sr_counts = Counter(sample_rates)
bps_counts = Counter(bit_depths)
ch_counts = Counter(channels)
print("Sample Rate Counts:", sr_counts)
print("Bit Depth Counts:", bps_counts)
print("Channel Counts:", ch_counts)


In [9]:
len(wav_info)